# Visale - Action Logement

La Garantie Visale est une aide octroyée par Action Logement.  
Elle est à destination des jeunes de moins de 30 ans ou des salariés nouvellement embauchés ayant besoin d'un cautionnement pour un logement.

* Descriptif officiel d'Action Logement : https://www.actionlogement.fr/la-garantie-visale 
* Site dédié à Visale d'Action Logement ; descriptif à destination des locataires : https://www.visale.fr/visale-pour-les-locataires/avantages/
* Fiche service-public.fr : https://www.service-public.fr/particuliers/vosdroits/F33453


In [174]:
# Active l'affichage de résultats multiples par cellule
# %matplotlib inline (pour des graphes)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Situation à analyser

### Cas de test d'aides-simulateur-front

In [175]:
situation_18yo_moving_away = {
    "id": "18yo-moving-away",
    "description": "Un jeune de 18 ans déménage pour des études à l'université",
    "answers": {
        "statut-professionnel": "etudiant",  # activite
        "situation-professionnelle": "sans-emploi",  # dispatchSituationProfessionnelle
        "etudiant-mobilite": "parcoursup-nouvelle-region",  # dispatchEtudiantMobilite
        "boursier": True,  # boursier
        "date-naissance": "2007-03-01",  # date_naissance
        "handicap": False,  # handicap
        "statut-marital": "celibataire",  # statut_marital
        "code-postal-nouvelle-ville": "75101",  # depcom
        "situation-logement": "locataire",  # dispatchSituationLogement
        "type-logement": "logement-meuble",  # dispatchTypeLogement
        "logement-conventionne": True,  # logement_conventionne
        "colocation": False,  # coloc
        "logement-parente-proprietaire": False,  # proprietaire_proche_famille
        "nombre-personnes-logement": 1,  # 🔥 exclude: True
        "loyer-montant-mensuel": 700,  # loyer
        "loyer-montant-charges": 100,  # charges_locatives
        "loyer-difficile-payer": True,  # exclude: True
        "type-revenus": [
            "aucun-autres-revenus"
        ],
        "confirmation-end": [
            "confirmation-end-oui"
        ]
    },
    "questionsToApi": [
        "locapass-eligibilite",
        "mobilite-master-1",
        "mobilite-parcoursup",
        "aide-personnalisee-logement",
        "garantie-visale-eligibilite",
        "garantie-visale"
    ],
    "results": {
        "locapass": 1200,
        "locapass-eligibilite": True,
        "mobilite-master-1": 0,
        "mobilite-master-1-eligibilite": False,
        "mobilite-parcoursup": 500,
        "mobilite-parcoursup-eligibilite": True,
        "aide-personnalisee-logement": 327,
        "aide-personnalisee-logement-eligibilite": True,
        "garantie-visale": 800,
        "garantie-visale-eligibilite": True
    }
}

to_test = situation_18yo_moving_away

In [176]:
# {
#     "statut-professionnel":"etudiant",
#     "situation-professionnelle":"sans-emploi",
#     "etudiant-mobilite":"pas-de-mobilite",  #
#     "date-naissance":"2000-12-13",
#     "statut-marital":"celibataire",
#     "code-postal-nouvelle-ville":"93048",
#     "situation-logement":"locataire",
#     "type-revenus":[
#         "revenus-activite"
#         ],
#     "salaire-imposable":10000000000000000,
#     "type-logement":"logement-non-meuble",
#     "logement-conventionne":false,
#     "colocation":false,
#     "logement-parente-proprietaire":false,
#     "nombre-personnes-logement":1,
#     "loyer-montant-mensuel":800,
#     "loyer-montant-charges":100
# }

from os import getcwd
from os.path import join


test_path_json_etudiant_1er_decile = join(getcwd(), '..', 'payloads', 'etudiant_1er_decile_request.json')
test_path_json_etudiant_10eme_decile = join(getcwd(), '..', 'payloads', 'etudiant_10eme_decile_request.json')


### Transcription du cas de test au format de l'API web

In [177]:
from utils_calculate import period, period_last_month  # mois actuel, mois précédent
from notebooks.utils_mapping_simulateur import format_to_openfisca_json

# si input aides-simplifiees
# situation_garantie_visale = format_to_openfisca_json(to_test, period)

# si chargement de json 
import json


to_test_path_json = test_path_json_etudiant_10eme_decile
with open(to_test_path_json, 'r', encoding='utf-8') as fichier:
    print(f"Chargement du test : {to_test_path_json}")
    situation_garantie_visale = json.load(fichier)


# pour vérifier le json chargé
# from json import dumps
# print(dumps(situation_garantie_visale, indent=2))


Chargement du test : /Users/sch/dev/gitruc/aides-calculatrice-back/notebooks/../payloads/etudiant_10eme_decile_request.json


## Initialisation de la simulation

In [178]:
from utils_calculate import france_tax_benefit_system, new_simulation

simulation = new_simulation(france_tax_benefit_system, situation_garantie_visale)
simulation.trace = True


## Comparaison des résultats de la simulation et des résultats attendus

In [179]:
from notebooks.utils_calculate import (
    est_dans_intervalle, 
    est_strictement_superieur_seuil,
    est_inferieur_ou_egal_plafond,
    est_enum_dans_liste,
    est_pas_enum_dans_liste, 
    date_de_moins_de_nb_mois,
    non_renseignee_ou_egale_valeur_par_defaut
    )

### Qui peut bénéficier de la garantie visale ?

Le locataire entrant dans un logement du parc locatif privé peut : 

* **Être âgé de 30 ans et moins** (jusqu’au 31ème anniversaire) : quelle que soit sa situation professionnelle y compris étudiant et alternant,
* **Être âgé de plus de 31 ans et** : 
  - Être salarié d’une entreprise du secteur privé et justifier d’un salaire mensuel net inférieur ou égal à 1 500 €,
  - Être salarié du secteur privé en mutation professionnelle : CDI en période d’essai ou CDD depuis moins de 6 mois, en promesse d’embauche ou en mutation. 
* **Quel que soit l’âge du locataire, faire l’objet d’une signature d’un bail mobilité** : Le [bail mobilité](https://www.anil.org/bail-mobilite/) est un nouveau contrat de location allant de 1 à 10 mois, signé entre le propriétaire d'un logement meublé et certains locataires (étudiants, salariés en mission temporaire ou formation professionnelle).

> Source : https://www.actionlogement.fr/la-garantie-visale
> Non modélisé : VISALE pour les contrats saisonniers (Nouveauté)

#### Cas 30 ans et moins, toute situation professionnelle

In [180]:
# seuil bas d'âge à 18 ans 
# indiqué sur cette page : https://www.visale.fr/visale-pour-les-locataires/eligibilite/
# mais non précisé sur https://www.actionlogement.fr/la-garantie-visale
# et absent de https://www.service-public.fr/particuliers/vosdroits/F33453
# TODO supprimer le seuil bas de 18 ans sauf pour les contrats saisonniers ( à modéliser, nouveauté d'après https://www.actionlogement.fr/la-garantie-visale ) ?
#
# seuil haut d'âge à 30 ans "jusqu'au 31ème anniversiare"
# indiqué par https://www.actionlogement.fr/la-garantie-visale

cas_1_condition_age = est_dans_intervalle(simulation, 'age', 18, 30, "Le locataire doit avoir entre 18 et 30 ans.", 0)

✅ Le locataire doit avoir entre 18 et 30 ans.
age  :  24


In [181]:
cas_1 = cas_1_condition_age
print("✅" if cas_1 else "❌", "L'individu est dans le 1er cas de la garantie visale.")

✅ L'individu est dans le 1er cas de la garantie visale.


#### Cas plus de 31 ans et salarié

In [182]:
if not cas_1:
    cas_2_condition_age = est_strictement_superieur_seuil(simulation, 'age', 30, "Le locataire a plus de 30 ans.", 0)

In [183]:
# CAS 2.1
# Être salarié d’une entreprise du secteur privé et justifier d’un salaire mensuel net inférieur ou égal à 1 500 €
# d'après https://www.actionlogement.fr/la-garantie-visale

from openfisca_france.model.base import TypesCategorieSalarie
from openfisca_france.model.revenus.activite.salarie import TypesContrat

if not cas_1:
    cas2_condition_salarie_secteur_prive = est_enum_dans_liste(
        simulation,
        'categorie_salarie',
        [
            TypesCategorieSalarie.prive_non_cadre, 
            TypesCategorieSalarie.prive_cadre
        ],
        TypesCategorieSalarie.names,
        "Le locataire est salarié d'une entreprise du secteur privé."
        )


    # salarié gagnant jusqu’à 1500€ nets/mois
    # INFO : à la démarche de demande de visale, la dernière fiche de paie est demandée => période = mois précédant la demande 
    cas_2_condition_salaire_net_mois_precedent = est_inferieur_ou_egal_plafond(
        simulation,
        'salaire_net',
        period_last_month,
        1500,
        "Le locataire a un salaire allant jusqu’à 1500€ nets/mois."
    )




In [184]:
# CAS 2.2
# Être salarié du secteur privé en mutation professionnelle : CDI en période d’essai ou CDD depuis moins de 6 mois, en promesse d’embauche ou en mutation. 
from notebooks.utils_calculate import affiche_resultat

if not cas_1:
    cas_2_condition_avoir_contrat_travail = est_pas_enum_dans_liste(
        simulation,
        'contrat_de_travail_type',
        [
            TypesContrat.cdi.name,  # TypesContrat.cdi est valeur par défaut de contrat_de_travail_type 
            TypesContrat.aucun.name, TypesContrat.formation.name  # != locataire embauché
        ],
        TypesContrat.names,
        "Le locataire a un contrat de travail hors CDI confirmé." # CDI période d'essai OK / confirmé KO
        )


    # embauché depuis moins de 6 mois

    cas_2_condition_duree_contrat_travail = date_de_moins_de_nb_mois(
        simulation,
        'contrat_de_travail_debut',  # 01/01/1870 par défaut :-D
        6,
        "Le locataire a un contrat de travail depuis moins de 6 mois.")

    cas_2_condition_contrat_de_travail_en_cours = non_renseignee_ou_egale_valeur_par_defaut(
        france_tax_benefit_system,
        simulation,
        'contrat_de_travail_fin',
        "Le locataire n'a pas de date de fin de contrat de travail (connue)."
        )


    # contrat de travail actuel ? (contrôle date début et date de fin)
    print("======= Bilan")

    cas_2_condition_embauche_moins_6mois = cas_2_condition_duree_contrat_travail * cas_2_condition_contrat_de_travail_en_cours
    duree_depuis_debut_contrat_travail = simulation.calculate('contrat_de_travail_debut', period).astype('timedelta64[M]')
    affiche_resultat(
        "contrat de travail actuel",
        "Le locataire est embauché depuis moins de 6 mois.",
        cas_2_condition_embauche_moins_6mois[0],
        f"depuis {duree_depuis_debut_contrat_travail[0]} mois" if duree_depuis_debut_contrat_travail[0] > 0 else "inconnu"  # "inconnue" pour palier à durée négative si date début non renseignée (01/01/1870 par défaut)
        )


    # TODO vérifier cohérence avec activite == actif ?

In [185]:
# en mobilité professionnelle
# = "faire la demande de la garantie Visale dans les 6 mois qui suivent la mutation (changement de lieu de travail dans la même entreprise ou le même groupe)"
# Src : https://www.service-public.fr/particuliers/vosdroits/F33453

# Justificatif de la mutation (avenant au contrat de travail, attestation de mutation
# de l’employeur, certificat de mutation de l’employeur, lettre de l’employeur,
# attestation de déménagement de l’entreprise). Le contrat de travail initial n’est
# pas demandé.
# Src : https://www.visale.fr/wp-content/uploads/2020/04/Visale-Ressources-Locataire_2020.pdf#page=4

# mutation au mois précédant la demande d'après :
# "Salariés en mutation | Dernière fiche de paie"
# Src : https://www.visale.fr/wp-content/uploads/2020/04/Visale-Ressources-Locataire_2020.pdf#page=5

if not cas_1:
    mobilite_professionnelle = True  # TODO ajouter à openfisca-france et au questionnaire aides-simplifiées
    cas_2_condition_mobilite_professionnelle = mobilite_professionnelle
    affiche_resultat(
        "[⚠️ non calculé] en mobilité professionnelle",
        "Le locataire est en mobilité professionnelle (justifiable par fiche de paie du mois précédent).",
        cas_2_condition_mobilite_professionnelle,
        mobilite_professionnelle
        )

In [186]:
# TODO distinguer le cas2 :
# salarié à <= 1500€ net
# du cas salarié en mutation pro

cas_2 = False
if not cas_1:
    cas_2 = cas_2_condition_age * cas2_condition_salarie_secteur_prive * cas_2_condition_avoir_contrat_travail  # hors CDI
    print("✅" if cas_2 else "❌", "L'individu est dans le 2ème cas de la garantie visale.")

In [187]:
# TODO Le locataire et le propriétaire (bailleur) ne doivent pas être membres de la même famille (grand-parent, parent, ou enfant).
# https://www.service-public.fr/particuliers/vosdroits/F33453

## Analyse des résultats

In [188]:
visale_variables = ['visale_eligibilite', 'visale_base_ressources', 'visale_montant_max']
# visale_variables_entities = ['menages', 'menages', 'menages']
# visale_variables_groupes = ['menage_1', 'menage_1', 'menage_1']

for variable_name in visale_variables:
    print(f"{variable_name}: {simulation.calculate(variable_name, period)}")


visale_eligibilite: [ True]
visale_base_ressources: [8.3333334e+15]
visale_montant_max: [1500.]


In [189]:
# TODO identifier la source des revenus
# visale_base_ressources = simulation.calculate('visale_base_ressources', period)  # inclut le salaire_net du mois précédent

In [190]:
age = simulation.calculate("age", period) 
print(f"age = {age}")
majeur = simulation.calculate("majeur", period)
print(f"majeur = {majeur}")
etudiant = simulation.calculate("etudiant", period)
print(f"etudiant = {etudiant}")
ressortissant_eee = simulation.calculate("ressortissant_eee", period)
print(f"ressortissant_eee = {ressortissant_eee}")
nationalite = simulation.calculate("nationalite", period)
print(f"nationalite = {nationalite}")
loyer = simulation.calculate("loyer", period)
print(f"loyer = {loyer}")
charges_locatives = simulation.calculate("charges_locatives", period)
print(f"charges_locatives = {charges_locatives}")
visale_montant_max = simulation.calculate("visale_montant_max", period)
print(f"visale_montant_max = {visale_montant_max}")
date_entree_logement = simulation.calculate("date_entree_logement", period)
print(f"date_entree_logement = {date_entree_logement}")

# eligibilite_loyer = (loyer + charges_locatives) <= visale_montant_max
# https://legislation.fr.openfisca.org/visale_eligibilite

age = [24]
majeur = [ True]
etudiant = [ True]
ressortissant_eee = [ True]
nationalite = [b'FR']
loyer = [800.]
charges_locatives = [100.]
visale_montant_max = [1500.]
date_entree_logement = ['2025-07-01']


###### test_path_json_etudiant_1er_decile

age = [24]
majeur = [ True]
etudiant = [ True]
ressortissant_eee = [ True]
nationalite = [b'FR']
loyer = [800.]
charges_locatives = [100.]
visale_montant_max = [800.]
date_entree_logement = ['2025-07-01']

> eligibilite_loyer = (loyer + charges_locatives) <= visale_montant_max
> mais ( 800 + 100 ) > 800
> d'où l'inéligibilité de l'étudiant

###### test_path_json_etudiant_10eme_decile

age = [24]
majeur = [ True]
etudiant = [ True]
ressortissant_eee = [ True]
nationalite = [b'FR']
loyer = [800.]
charges_locatives = [100.]
visale_montant_max = [1500.]
date_entree_logement = ['2025-07-01']

> eligibilite_loyer = (loyer + charges_locatives) <= visale_montant_max
> mais ( 800 + 100 ) <= 1500
> d'où l'éligibilité

In [191]:
affiche_resultat(
    'éligibilité obtenue', 
    "L'éligibilité attendue est égale au résultat obtenu par simulation.", 
    to_test["results"]["garantie-visale-eligibilite"] == simulation.calculate("visale_eligibilite", period)[0],
    simulation.calculate("visale_eligibilite", period)[0]
    )


affiche_resultat(
    'montant max obtenu', 
    "Le montant max attendu est égal au montant obtenu par simulation.", 
    to_test["results"]["garantie-visale"] == simulation.calculate("visale_montant_max", period)[0],
    simulation.calculate("visale_montant_max", period)[0]
    )


✅ L'éligibilité attendue est égale au résultat obtenu par simulation.
éligibilité obtenue  :  True
❌ Le montant max attendu est égal au montant obtenu par simulation.
montant max obtenu  :  1500.0


In [ ]:
from notebooks.utils_calculate import (
    est_enum_dans_liste,
    )

In [193]:
# en possession d’une promesse d’embauche de moins de 3 mois

# "Futur salarié | Promesse d’embauche(*) ou contrat de travail"
# (*) La forme de la promesse d’embauche est libre dès que l’employeur peut être identifié et son intégrité établie. Pour ce faire,
# celle-ci doit contenir les informations relatives à la date de la promesse d’embauche, au salaire, aux conditions du contrat de
# travail (CDD, CDI etc.), à la durée de l’emploi, à la date d’entrée dans l’emploi et à l’employeur (Siret etc.). Une promesse
# d’embauche sous format dématérialisé est acceptée.
# Src : https://www.visale.fr/wp-content/uploads/2020/04/Visale-Ressources-Locataire_2020.pdf#page=4

contrat_de_travail_debut = simulation.calculate('contrat_de_travail_debut', period)
duree_depuis_debut_contrat_travail = contrat_de_travail_debut.astype('timedelta64[M]')
contrat_de_travail_debut_default_value = france_tax_benefit_system.get_variable('contrat_de_travail_debut').default_value

promesse_embauche_moins_3mois = True  # TODO ajouter à openfisca-france et au questionnaire aides-simplifiées
cas_2_condition_promesse_embauche = (
    contrat_de_travail_debut != contrat_de_travail_debut_default_value  # avoir un début de contrat de travail défini
  ) * promesse_embauche_moins_3mois

affiche_resultat(
    "[⚠️ non calculé] promesse d'embauche",
    "Le locataire dispose d'une promesse d'embauche de moins de 3 mois.",
    cas_2_condition_promesse_embauche[0],
    cas_2_condition_promesse_embauche[0]
    )

❌ Le locataire dispose d'une promesse d'embauche de moins de 3 mois.
[⚠️ non calculé] promesse d'embauche  :  False


#### Cas 3 quelque soit l'âge, je suis éligible au bail mobilité.

> Source : https://www.visale.fr/visale-pour-les-locataires/eligibilite/

Le bail mobilité est un bail signé entre le propriétaire (bailleur) d'un logement meublé 
et un locataire considéré comme occupant temporaire (étudiant, salarié en mission temporaire ou en formation professionnelle...). 
D'une durée allant de 1 à 10 mois, il ne peut pas être renouvelé.

Pour quel locataire ?

Seules certaines personnes peuvent être le locataire (ou colocataire) d'un bail mobilité.  
Lorsque le bail débute (date qui peut être différente de celle de la signature du bail), le locataire doit se trouver dans l'une des situations suivantes :

* Formation professionnelle
* Études supérieures
* Contrat d'apprentissage
* Stage
* Engagement volontaire dans le cadre d'un service civique
* Mutation professionnelle
* Mission temporaire dans le cadre de l'activité professionnelle

> Source : https://www.service-public.fr/particuliers/vosdroits/F34759

In [194]:
# TypesContrat.formation ou ContexteActivitePoleEmploi.formation (et TypesActivite.actif par cohérence ?)
# TypesActivite.etudiant
# apprentissage_contrat_debut == True ou apprenti == True
# stagiaire == True (et ajouter contrat_de_travail_type à cdd ou cdi ? ou ajouter catégorie de contrat à TypesContratTravailDureeDeterminee et compter CDI standard ?)
# service_civique == True
# = mobilité professionnelle cas +30 and ci-dessous ?
# ajouter mission à TypesContrat ? mais cumulable avec autre contrat donc distinguer dans une nouvelle catégorie ?

eligible_bail_mobilite = False  # TODO ajouter à openfisca-france et au questionnaire aides-simplifiées
cas_2_condition_eligible_bail_mobilite = eligible_bail_mobilite
affiche_resultat(
    "[⚠️ non calculé] éligible au bail mobilité",
    "Le locataire est éligible au bail mobilité.",
    cas_2_condition_eligible_bail_mobilite,
    eligible_bail_mobilite
    )

❌ Le locataire est éligible au bail mobilité.
[⚠️ non calculé] éligible au bail mobilité  :  False


### Éligibilité logement

* Peut être meublé ou non
* Doit être ma résidence principale
* Doit avoir un loyer (charges comprises) de 1500€ maximum en Île de France ou de 1300€ maximum sur le reste du territoire
* Doit être décent et respecter le Règlement Sanitaire Départemental (RSD)
* Doit faire l’objet d’un bail (contrat de location)


> Source : https://www.visale.fr/visale-pour-les-locataires/eligibilite/

In [195]:
# peut être meublé ou non
from openfisca_france.model.base import TypesStatutOccupationLogement


# logement possiblement futur mais simulation sur période actuelle 
condition_type_logement = est_enum_dans_liste(
    simulation,
    'statut_occupation_logement',
    [
        TypesStatutOccupationLogement.locataire_meuble.name,
        TypesStatutOccupationLogement.locataire_vide.name
        # TODO + locataire_foyer + locataire_hlm ?
    ],
    TypesStatutOccupationLogement.names,
    "Le logement est meublé ou non."
    )


✅ Le logement est meublé ou non.
statut_occupation_logement  :  locataire_vide


In [196]:
# doit être ma résidence principale

In [197]:
# doit avoir un loyer (charges comprises) de 1500€ maximum en Île de France ou de 1300€ maximum sur le reste du territoire

In [198]:
# doit être décent et respecter le Règlement Sanitaire Départemental (RSD)

In [199]:
# doit faire l’objet d’un bail (contrat de location)

## Limites

> Source : https://www.visale.fr/wp-content/uploads/2024/08/depliant_visale_locataire_juin24.pdf

Dans la limite de 36 mensualités d’impayés de loyers
et charges pour un logement du parc privé. 
Dans le parc locatif social ou assimilé, prise en charge dans la limite de 9
mois d’impayés de loyers et charges locatives, déduction faite
des aides au logement. 
Dans le parc locatif privé, en cas de
dégradations imputables au locataire, les frais de remise en
état du logement sont couverts, jusqu'à deux mois de loyer et
charges inscrits au bail, en complément du dépôt de garantie
(sauf pour le bail mobilité).

Pour les travailleurs saisonniers, la durée de prise en charge
des impayés est limitée de 1 à 9 mois quel que soit le parc
locatif. Pour les dégradations locatives : prise en charge jusqu’à
deux mois de loyer et charges inscrits au bail, complétée le
cas échéant d’une prise en charge des dommages mobiliers, à
hauteur d’un mois de loyer.

In [200]:
simulation.tracer.print_computation_log()

In [201]:
# with open('debug.py', 'w') as fichier:
#     fichier.write(repr(simulation.tracer._trees))